In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

### RDD 생성

In [3]:
marks=[
    "'김하나','English', 100",
    "'김하나','Math', 80",
    "'임하나','English', 70",
    "'임하나','Math', 100",
    "'김갑돌','English', 82.3",
    "'김갑돌','Math', 98.5"
]

# 2차원 data에서 rdd만드려면 parallelize사용
_marksRdd=spark.sparkContext.parallelize(marks)

### 문제 3-1
이름으로 합계를 구해보자. 이름과 점수로 데이터를 추출하고, 이름별로 (이름을 키로) 합계를 계산한다.

개인별 데이터를 컴마로 분리하고, 이름 x[0]과 성적 x[2]만 꺼내어 reduceBykey()를 구하면 합계를 구할 수 있다. 여기서 중요한 것은 기존 데이터에서 필요한 이름, 성적만을 꺼내어 처리한다는 점이다.

In [12]:
# marks by name
_marksbyname=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))\ # 3 -> 2column으로 바꿈
    .reduceByKey(lambda x,y:x+y)\
    .collect()

In [13]:
for i in _marksbyname:
  print (i[0],i[1])

'김하나' 180.0
'임하나' 170.0
'김갑돌' 180.8


### 문제 3-2
과목으로 합계를 계산해 보자. 과목과 점수로 데이터를 추출하여, 과목별로 (과목을 키로) 합계를 계산한다.

개인별 데이터를 컴마로 분리하고, 과목 x[1]과 성적 x[2]만 꺼내어 reduceBykey()를 구하면 합계를 구할 수 있다.

In [6]:
# marks by subject
_marksbysubject=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[1],float(x[2])))\
    .reduceByKey(lambda x,y:x+y)\
    .collect()
for i in _marksbysubject:
  print (i[0],i[1])

'English' 252.3
'Math' 278.5


### 문제 3-3
이름으로 합계와 개수를 구해보자. 이름과 점수로 데이터를 추출하여, 이름별로 (이름을 키로) 합계와 개수를 계산한다.

합계, 개수를 계산해 보자. combineByKey()를 이용해서 계산해야 한다. 먼저 데이터를 이름, 과목, 데이터 -> 이름, 점수로 변경한다


In [7]:
# marks by name
_marksbyname2=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))

In [8]:
# sum, counts by name
sum_counts = _marksbyname2.combineByKey(
    (lambda x: (x, 1)), # the initial value, with value x and count 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # how to combine a pair value with the accumulator: sum value, and increment count
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)

In [9]:
for i in sum_counts.collect():
    for each in i:
        print (each, end=' ')
    print()

'김하나' (180.0, 2) 
'임하나' (170.0, 2) 
'김갑돌' (180.8, 2) 


### 문제 3-4
이름으로 평균을 계산해 보자. 

개인별 평균은 3-3에서 구했던 합계, 개수를 사용하여 계산한다. 평균을 계산하기 위해 float() 형변환을 해주었다.

In [10]:
# average
averageByKey = sum_counts\
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .collect()
for i in averageByKey:
    for j in i:
        print (j, end=' ')
    print()

'김하나' 90.0 
'임하나' 85.0 
'김갑돌' 90.4 
